In [2]:
import pandas as pd
import numpy as np

In [3]:
dataset1 = pd.read_csv('dalas/project/githubProject/ConflictPrediction/dataset1/dataset1_beforeSplit.csv')


# TRAITEMENT SUR LE DATASET 1 (MASKED)

In [4]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


## Code pour split le dataset 1 en train/test 90-10 %

X = dataset1.drop(columns=['war'])
y = dataset1['war']

# Split train/test final
X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.1,# 10% pour test
    stratify=y, #pour garder la répartition des classes
    random_state=42 #pour mettre une random seed fixe
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)



Train shape: (6610, 221)
Test shape: (735, 221)


In [5]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# stratified K-fold cross-validation
def pipeline_stratifiedKFold_and_training(X_train,y_train,k=10):
    list_f1_scores = []
    skf = StratifiedKFold(n_splits=k, shuffle=True,random_state=42) # shuffle pour mélanger les données avant de split 

    for i,(train_index,val_index) in enumerate(skf.split(X_train, y_train)):

        # On récupère les sous-datasets d'entraînement et de validation
        X_tr = X_train.iloc[train_index]
        X_val=  X_train.iloc[val_index]
        y_tr  = y_train.iloc[train_index] 
        y_val = y_train.iloc[val_index]

        print(f"Fold {i}")
        # Maintenant on entraine et évalue le modèle avec X_tr, y_tr, X_val, y_val

        #### À REVOIR ###

        # Modèle XGBoost
        model = xgb.XGBClassifier(# Hyperparamètres à voir
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            random_state=42
        )
        # Entraînement
        model.fit(X_tr, y_tr)

        #Evaluation
        y_pred = model.predict(X_val)
         # Métriques
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred)
        rec = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        list_f1_scores.append(f1)
        #print(f"Accuracy : {acc:.4f}")
        #print(f"Precision: {prec:.4f}")
        #print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    avg_f1 = np.mean(list_f1_scores)
    print(f"\nAverage F1-score over {k} folds: {avg_f1:.4f}")
    std_f1 = np.std(list_f1_scores)
    print(f"Standard Deviation of F1-score over {k} folds: {std_f1:.4f}")

pipeline_stratifiedKFold_and_training(X_train, y_train, k=10)

Fold 0
F1-score : 0.9495
Fold 1
F1-score : 0.9582
Fold 2
F1-score : 0.9664
Fold 3
F1-score : 0.9472
Fold 4
F1-score : 0.9504
Fold 5
F1-score : 0.9502
Fold 6
F1-score : 0.9565
Fold 7
F1-score : 0.9516
Fold 8
F1-score : 0.9446
Fold 9
F1-score : 0.9562

Average F1-score over 10 folds: 0.9531
Standard Deviation of F1-score over 10 folds: 0.0060
